In [34]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import math

In [3]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

# Функции для определения характеристик квартиры. Цена:

In [6]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

# Координаты

In [56]:
def getDist(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    # Расстояние по Пифагору (цетр Москвы: 55.753639, 37.619833):
    Dist = math.sqrt((lat - 55.753639)**2 + (lon- 37.619833)**2)
    return Dist

# Число комнат

In [8]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    return room_number

# Параметры площади

In [9]:
def get_square(flat_page):
    p = flat_page.find('div', attrs={'class':'clearfix'})
    p_tot = re.split('Общая площадь|комнат', str(p))[1]
    p_tot = re.split('</i>|\xa0м<sup>', str(p_tot))[1]
    Totsp = p_tot.replace(',', '.')
    p_live = re.split('Жилая|кухни', str(p))[1]
    p_live = re.split('</i>|\xa0м<sup>', str(p_live))[1]
    Livesp = p_live.replace(',', '.')
    p_kit = re.split('кухни|сан', str(p))[1]
    p_kit = re.split('</i>|\xa0м<sup>', str(p_kit))[1]
    Kitsp = p_kit.replace(',', '.')
    return Totsp, Livesp, Kitsp

# Сколько и как добираться до метро

In [10]:
def GetMetro(flat_page):
    p = flat_page.find('div', attrs={'class':'object_descr_metro'})
    p = re.split('comment', str(p))
    try:
        p = p[1]
        Metrdist = [i for i in p.split() if i.isdigit()]
        if re.search('пеш', p) is not None:
            Walk = 1
        else:
            Walk = 0
    except: 
        Metrdist = 'N'
        Walk = 0
    return Metrdist, Walk


# Номер этажа, количество этажей, балконы

In [11]:
def GetBalc(flat_page):
    p = flat_page.find('div', attrs={'class':'clearfix'})
    p = re.split('Балкон|Лифт', str(p))[1]
    if re.search('балк|лодж', str(p)) is not None:
            Bal = 1
    else:
            Bal = 0
    return Bal

In [12]:
def GetFloors(flat_page):
    floor = flat_page.find('div', attrs={'class':'clearfix'})
    floor = re.split('Этаж|Тип', str(floor))
    floor = floor[1].split()
    floor = [i for i in floor if i.isdigit()]
    try :
        Floor = int(floor[0])
    except:
        Floor = 'N'
    try:
        Nfloors = int(floor[1])
    except:
        Nfloors = 'N'
    return Floor, Nfloors

# Тип дома (кирпичный, вторичка)

In [13]:
def GetType(flat_page):
    p = flat_page.find('div', attrs={'class':'clearfix'})
    p = re.split('дома|Высота', str(p))
    try:
        p = p[1]
        if re.search('втор', str(p)) is None:
            New = 1
        else:
            New = 0
        if re.search('кирп|монолит|жб', str(p)) is not None:
            Brick = 1
        else:
            Brick = 0
    except:
        Brick = 'N'
        New = 'N'
    return Brick, New

# Объединяем информацию:

In [51]:
flatStats = {'District':0}
def get_stats(flat_page):
    flatStats['Totsq'] = get_square(flat_page)[0]
    flatStats['Livesq'] = get_square(flat_page)[1]
    flatStats['Kitsq'] = get_square(flat_page)[2]
    flatStats['Price'] = getPrice(flat_page)
    flatStats['Rooms'] = getRoom(flat_page)
    flatStats['Bal'] = GetBalc(flat_page)
    flatStats['Floor'] = GetFloors(flat_page)[0]
    flatStats['NFloors'] = GetFloors(flat_page)[1]
    flatStats['Metrodist'] = GetMetro(flat_page)[0]
    flatStats['Walk'] = GetMetro(flat_page)[1]
    flatStats['Brick'] = GetType(flat_page)[0]
    flatStats['New'] = GetType(flat_page)[1]
    flatStats['Dist'] = getDist(flat_page)
    return flatStats
    

# Результаты. ЦАО:

In [15]:
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [16]:
links = []
for page in range(1, 30):
    page_url =  district.format(page)

    search_page = requests.get(page_url)
    search_page = search_page.content
    search_page = BeautifulSoup(search_page, 'lxml')

    flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
    flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

    for link in flat_urls:
        if link.isdigit():
            links.append(link)

In [19]:
CIAN_flats_CAO = pd.DataFrame()
for i in links:
        flat_url = 'http://www.cian.ru/sale/flat/' + str(i) + '/'
        #flat_url = 'http://www.cian.ru/sale/flat/150531912/' 
        flat_page = requests.get(flat_url)
        flat_page = flat_page.content
        flat_page = BeautifulSoup(flat_page, 'lxml')
        results = get_stats(flat_page)
        results['District'] = 0
        CIAN_flats_CAO = CIAN_flats_CAO.append(results, ignore_index=True)

# Остальные округа Mосквы

Я разбивала на куски, чтобы не ввергнуть свой комп в шок)
Загружаем вручную адреса округов:

In [42]:
district = ['http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1','http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B1%5D=40&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p=30&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1', 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'] 

Сначала первые пять:

In [43]:
CIAN_flats_5 = pd.DataFrame()

In [ ]:
links = []
for d in range(1,5):
    for page in range(1, 30):
        page_url =  district[d].format(page)
        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')
        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))
        for link in flat_urls:
            if link.isdigit():
                links.append(link)
        for i in links:
            flat_url = 'http://www.cian.ru/sale/flat/' + str(i) + '/'
            #flat_url = 'http://www.cian.ru/sale/flat/150531912/' 
            flat_page = requests.get(flat_url)
            flat_page = flat_page.content
            flat_page = BeautifulSoup(flat_page, 'lxml')
            results = get_stats(flat_page)
            results['District'] = d
        
            CIAN_flats_5 = CIAN_flats_5.append(results, ignore_index=True)

И последние пять:

In [58]:
CIAN_flats_10 = pd.DataFrame()

In [ ]:
links = []
for d in range(6,11):
    for page in range(1, 30):
        page_url =  district[d].format(page)
        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')
        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))
        for link in flat_urls:
            if link.isdigit():
                links.append(link)
        for i in links:
            flat_url = 'http://www.cian.ru/sale/flat/' + str(i) + '/'
            #flat_url = 'http://www.cian.ru/sale/flat/150531912/' 
            flat_page = requests.get(flat_url)
            flat_page = flat_page.content
            flat_page = BeautifulSoup(flat_page, 'lxml')
            results = get_stats(flat_page)
            results['District'] = d
        
            CIAN_flats_10 = CIAN_flats_10.append(results, ignore_index=True)

Соединяем кусочки:

In [46]:
CIAN = pd.concat([CIAN_flats_CAO, CIAN_flats_5], ignore_index=True)

Редактируем конечную таблицу

In [47]:
for i, row in CIAN.iterrows():
    try:
        float(row[4])
        CIAN.set_value(i,'Kitsq',float(row[4]))
    except:
        CIAN.set_value(i,'Kitsq','N')
    try:
        float(row[5])
        CIAN.set_value(i,'Livesq',float(row[5]))
    except:
        CIAN.set_value(i,'Livesq','N')
    if row[6] == 'N':
        continue
    else:
        float(row[6][0])
        CIAN.set_value(i,'Metrodist',float(row[6][0]))


Создаем csv-файл

In [578]:
CIAN.to_csv('CIAN_parsing.csv', sep = ',')